In [1]:
from googleAPI import *
from pymongo import MongoClient
from crawling import ProgressBar
from pprint import pprint


In [2]:
client = MongoClient(
    "mongodb+srv://cluster0.c6ccx.mongodb.net",
    authsource="$external",
    authmechanism="MONGODB-X509",
    tls=True,
    tlsCertificateKeyFile="../../../mongodb.pem",
)
db = client["youtube"]
rawVideoSearch = db["rawVideoSearch"]
videoSearch = db["videoSearch"]
videoCategories = db["videoCategories"]
videoFavorite = db["videoFavorite"]


In [4]:
with open("apikey.txt", encoding="utf8") as f:
    apikey1 = f.readline().strip()
    apikey2 = f.readline().strip()


In [30]:
global_query = {"key": apikey2, "maxResults": 100, "regionCode": "KR"}


In [31]:
import datetime

# 1970-01-01T00:00:00Z
def datetime_to_string(datetime: datetime.datetime):
    return datetime.strftime("%Y-%m-%dT%H:%M:%SZ")


def string_to_datetime(str):
    return datetime.datetime().strptime(str, "%Y-%m-%dT%H:%M:%SZ")


# search with keyword(costs to much(100 times much than other))
def get_searched_video(query):
    init_query = {**global_query, "part": "id, snippet", "type": "video", "safeSearch": "strict"}
    for key in query:
        init_query[key] = query[key]
    return get_youtube_api_json("search", init_query)


In [ ]:
# get top view count video of korea from search
rawVideoSearch = db["rawVideoSearch"]
start_date = datetime_to_string(datetime.datetime(2019, 1, 1))
end_date = datetime_to_string(datetime.datetime(2020, 1, 1))
init_query = {"publishedAfter": start_date, "publishedBefore": end_date}
total_search_data = []
nextPageToken = ""
for i in range(20):
    init_query["pageToken"] = nextPageToken
    search_data = get_searched_video(init_query)
    total_search_data.append(search_data)
    nextPageToken = search_data.get("nextPageToken")
    if nextPageToken is None:
        print(search_data)
        break
rawVideoSearch.insert_one({"searchDate": datetime.datetime.now(),"publishedAfter": start_date, "publishedBefore": end_date, "result":total_search_data})


In [ ]:
videoSearch.find()


In [ ]:
# get category data of korea
videoCategories = db["videoCategories"]
category_data = get_youtube_api_json("videoCategories", {"key": apikey1, "part": "snippet, id", "regionCode": "KR"})[
    "items"
]
videoCategories.insert_many(category_data)


In [ ]:
list(videoCategories.find())


In [42]:
def get_populer_video(query):
    init_query = {**global_query,
        "part": "snippet, contentDetails, status, statistics, player, topicDetails, recordingDetails",
        "chart": "mostPopular",
    }
    for key in query:
        init_query[key] = query[key]
    return get_youtube_api_json("videos", init_query)


def get_video(query):
    init_query = {**global_query,
        "part": "snippet, contentDetails, status, statistics, player, topicDetails, recordingDetails"
    }
    for key in query:
        init_query[key] = query[key]
    return get_youtube_api_json("videos", init_query)


In [14]:
# videoSearch = db["videoSearch"]
# rawVideoSearch = db["rawVideoSearch"]
# rawVideoSearch.insert_many(videoSearch.find())
# videoSearch.drop()


In [ ]:
import re

hangul = re.compile("[ㄱ-ㅎ가-힣]+")
rawVideoSearch = db["rawVideoSearch"]
videoSearch = db["videoSearch"]
total_pre_data = []
for datas in rawVideoSearch.find():
    for data in datas["items"]:
        try:
            preData = get_video({"id": data["id"]["videoId"]})["items"][0]
        except:
            continue
        if len(hangul.findall(preData["snippet"]["title"])) > 0:
            print(preData["snippet"]["title"])
            total_pre_data.append(preData)
videoSearch.insert_one({"searchDate": datetime.datetime.now(), "info": total_pre_data})


In [ ]:
data = list(videoSearch.find())
pre_data = []
for i in data:
    for j in i["info"]:
        pre_data.append((int(j["statistics"]["viewCount"]), j["snippet"]["title"].strip()))
pre_data.sort(reverse=True)
pre_data


In [ ]:
for category_data in videoCategories.find():
    categoryId = category_data["id"]
    print(categoryId)


In [13]:
get_video({"id": "8Syy24YLOQw,Iim-a-dmND8,Ws57hamuOZk"})


{'kind': 'youtube#videoListResponse',
 'etag': 'fGXIqX0JkdCXzEOg44s5tY1lsmQ',
 'items': [{'kind': 'youtube#video',
   'etag': '6E1oWtTm91c2ngG2tDzP-vrE8pM',
   'id': '8Syy24YLOQw',
   'snippet': {'publishedAt': '2022-02-24T22:16:45Z',
    'channelId': 'UCsjzJWuBnPyA8-tj9pEDwXg',
    'title': '현대가 BMW를 처바르는 놀라운 광경을 보게 되다니! 제네시스 eGV70',
    'description': '#eGV70 #IX3 #제네시스 #BMW',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/8Syy24YLOQw/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/8Syy24YLOQw/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/8Syy24YLOQw/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/8Syy24YLOQw/sddefault.jpg',
      'width': 640,
      'height': 480},
     'maxres': {'url': 'https://i.ytimg.com/vi/8Syy24YLOQw/maxresdefault.jpg',
      'width': 1280,
      'height': 720}},
    'channelTit

In [43]:
start_date = datetime_to_string(datetime.datetime(2019, 1, 1))
end_date = datetime_to_string(datetime.datetime(2020, 1, 1))
rawVideoSearch = db["rawVideoSearch"]
videoSearch = db["videoSearch"]
total_data = []
for datas in rawVideoSearch.find_one({"publishedAfter": start_date, "publishedBefore": end_date})["result"]:
    data_id = []
    for data in datas["items"]:
        try:
            data_id.append(data["id"]["videoId"])
        except:
            continue
    total_data.append(get_video({"id":",".join(data_id)}))
videoSearch.insert_one({"searchDate": datetime.datetime.now(),"publishedAfter": start_date, "publishedBefore": end_date, "result":total_data})

eXBu09fwe3I,M9Uy0opVF3s,bqzDuRz_P7g,62E_xyj_oDA,NaKrke1EL1A,6yWPfUz0z94,i23NEQEFpgQ,cfm97EKin4c,-6xhgynDlYY,MOwaUlXZxkI,W3iSnJ663II,vXmVvAgzgVI,LTz8NjNQp-s,o4GHoqGtRkg,XsX3ATc3FbA,X-iJZ0gfKPo,uR8Mrt1IpXg,Eo_mo5vA7tw,X-aaDs5heQ4,iFoqGyWhMws,N5qWjQ9j6l0,-EfjXQgE1e8,4HG_CJzyX6A,ESVsbCkFvG4,XkDA02FHHik,-OfOkiVFmhM,vx9CSpnROfs,y3rj2CDHF5c,9yimV9i-dY0,LrsH9vWq0Xg,MkG24J87IUk,x95oZNxW5Rc,eP4ga_fNm-E,j2aQ_NqeTNw,SvV49SD99DY,CbFh06IpSwA,iVzFO-1euU8,8M146nXXTYg,pAnK1y7qjuE,AO1OqWwKj1A,AtNBhPxVwh0,57sYinArEC4,2S24-y0Ij3Y,UsaGgEjqNoE,FG0nTVU3fRM,irXZve4QRos,JrOrlhjIYVk,_E5JtAcc3cY,azr7saygKng,aNcgx832uMM
[{'kind': 'youtube#videoListResponse', 'etag': 'PoroHx5E4WlTH0sZjjRAC8r_vzc', 'items': [{'kind': 'youtube#video', 'etag': 'Po7syKrRg8SlGBYOSc1cjYN1fbg', 'id': 'eXBu09fwe3I', 'snippet': {'publishedAt': '2019-10-18T09:00:01Z', 'channelId': 'UC3IZKseVpdzPSBaWxBxundA', 'title': "BTS (방탄소년단) 'Make It Right (feat. Lauv)' Official MV", 'description': "BTS (방탄소년단) 'Make It Right (feat. Lauv)' Official MV\

In [10]:
videoFavorite = db["videoFavorite"]
total_data = []
for category_data in videoCategories.find():
    categoryId = category_data["id"]
    nextPageToken = ""
    for i in range(20):
        favorite_data = get_populer_video({"videoCategoryId": categoryId, "pageToken": nextPageToken})
        nextPageToken = favorite_data.get("nextPageToken")
        total_data.append({"searchDate": datetime.datetime.now(), "searchCategory": categoryId, "info": favorite_data})
        if nextPageToken is None:
            break
    break
print(total_data)
# videoFavorite.insert_many(totalData)


[{'searchDate': datetime.datetime(2022, 2, 28, 13, 3, 40, 739070), 'searchCategory': '1', 'info': {'kind': 'youtube#videoListResponse', 'etag': 'utBbZ4qdhV9A4UsvXPHdAiAWzic', 'items': [{'kind': 'youtube#video', 'etag': 'lJDtkw0SCUaWn1i9m2KtFozyiLM', 'id': 'vC-aS_VVgGQ', 'snippet': {'publishedAt': '2022-02-26T13:00:23Z', 'channelId': 'UCxn_EHVOJnrEJBZY2p3E6Sg', 'title': '촉법소년을 극혐하는 참교육 전문 판사에게 개겼다가 나락가버린 일진 양아치들. 넷플릭스 신작 드라마《소년심판》시즌1 한방에 보기', 'description': '안녕하세요^^\n채널 어바웃타임 입니다.\n오늘 소개해드릴 드라마는 소년심판 시즌1 입니다.\n\n-어바웃타임 부채널(노딱,삭제 및 결말 위주의 업로드채널 입니다)\nhttps://www.youtube.com/channel/UCylVDnjEbLkFG4pZb7CqFkw\n\n#소년심판 #김혜수 #김무열 #이성민 #촉법소년 #넷플릭스', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/vC-aS_VVgGQ/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/vC-aS_VVgGQ/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/vC-aS_VVgGQ/hqdefault.jpg', 'width': 480, 'height': 360}, 'standard': {'url': 'https://i.ytimg.co

In [ ]:
init_data = get_populer_video()
total_results = init_data["pageInfo"]["totalResults"]
print(total_results)
# nextPageToken = init_data.get("nextPageToken")
# populer_video_data = [init_data]
# for i in range(5):
#     print(nextPageToken)
#     data = get_populer_video(nextPageToken)
#     populer_video_data.append(data)
#     nextPageToken = data.get("nextPageToken")
#     if nextPageToken is None:
#         break
# pprint(populer_video_data)


In [ ]:
get_youtube_api_json(
    "commentThreads",
    {**global_query,
        "part": "id, replies, snippet",
        "textFormat": "plainText",
        "videoId": "zWhqy34zvVI",
        "order": "relevance",
        # "searchTerms":"차준환"
    },
)
